In [1]:
!pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.44.0.2-py2.py3-none-any.whl size=265363806 sha256=688b35198036dc2fe8cf37ac297ca9d6d84a602a506c6cc6a90051df7d3443ab
  Stored in directory: /root/.cache/pip/wheels/e9/ea/ab/889709967692e30c57c46137edd2cae6b2768bb65ec7cb8aa5
Successfully built h2o


In [62]:
import pandas as pd
import h2o
from sklearn.model_selection import train_test_split
from h2o.automl import H2OAutoML
from sklearn.metrics import mean_squared_error

In [63]:
TOX = pd.read_csv("interaction_data.csv")

In [64]:
TOX.head()

,Halogens,S,P,Other Metal,Hydroxyl or Ether,"Ketones, aldehydes, carboxylic acids, or esters",Primary Amine or Ammonium Ion,Secondary Amine or Aromatic Nitrogen,Tertiary Amine or Amide Nitrogen,Aromatic,...,Primary_Amine_Ketones_aldehydes_acids_esters,Secondary_Amine_Ketones_aldehydes_acids_esters,Tertiary_Amine_Ketones_aldehydes_acids_esters,Primary_Amine_Hydroxyl_or_Ether,Secondary_Amine_Hydroxyl_or_Ether,Tertiary_Amine_Hydroxyl_or_Ether,Aromatic_Hydroxyl_or_Ether,Aromatic_Other_Metal,Hydroxyl_or_Ether_Ketones_aldehydes_acids_esters,Aliphatic_Tertiary_Amine_or_Amide_Nitrogen
0,0,0,0,0,1,-1,0,0,0,1,...,0,0,0,0,0,0,1,0,-1,0
1,0,1,1,0,0,0,-1,-1,1,0,...,0,0,0,0,0,0,0,0,0,1
2,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,-1,0,0,0,1,...,0,0,0,0,0,0,1,0,-1,0


### Full Model

In [194]:
Xtrain = TOX.drop(columns=['mouse intraperitoneal LD50',
                           'Tertiary_Amine_Ketones_aldehydes_acids_esters',
                           'Primary_Amine_Ketones_aldehydes_acids_esters',
                           'Secondary_Amine_Hydroxyl_or_Ether',
                           'Tertiary_Amine_Hydroxyl_or_Ether',
                           'Secondary_Amine_Ketones_aldehydes_acids_esters',
                           'Hydroxyl or Ether'])
ytrain = TOX['mouse intraperitoneal LD50']

# Split the data into training (70%), validation (15%), and test (15%) sets
X_train, X_temp, y_train, y_temp = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [195]:
# H2O.ai
h2o.init()

Xtrain_h2o = h2o.H2OFrame(X_train)
Xtest_h2o = h2o.H2OFrame(X_test)
X_valid_h2o = h2o.H2OFrame(X_valid)
ytrain_h2o = h2o.H2OFrame(list(y_train), column_names=["mouse intraperitoneal LD50"])
ytest_h2o = h2o.H2OFrame(list(y_test), column_names=["mouse intraperitoneal LD50"])
yvalid_h2o = h2o.H2OFrame(list(y_valid), column_names=["mouse intraperitoneal LD50"])

train_h2o = Xtrain_h2o.cbind(ytrain_h2o)
test_h2o = Xtest_h2o.cbind(ytest_h2o)
valid_h2o = X_valid_h2o.cbind(yvalid_h2o)

aml = H2OAutoML(max_runtime_secs=3600, max_models=10, seed=42, nfolds=5)
aml.train(y="mouse intraperitoneal LD50", training_frame=train_h2o, validation_frame=valid_h2o)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,3 hours 8 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_unknownUser_lvn68v
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.579 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
00:18:29.192: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),9/10
# GBM base models (used / total),4/4
# XGBoost base models (used / total),3/3
# DRF base models (used / total),1/2
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [201]:
# View AutoML leaderboard
leaderboard = aml.leaderboard
print(leaderboard)

model_id                                                     rmse       mse       mae     rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_22_20231127_01829     0.621751  0.386575  0.443087  0.147777                  0.386575
StackedEnsemble_BestOfFamily_1_AutoML_22_20231127_01829  0.621954  0.386827  0.442966  0.147822                  0.386827
GBM_4_AutoML_22_20231127_01829                           0.622707  0.387764  0.443295  0.147967                  0.387764
GBM_3_AutoML_22_20231127_01829                           0.623133  0.388294  0.444435  0.148071                  0.388294
GBM_2_AutoML_22_20231127_01829                           0.624976  0.390594  0.446388  0.148451                  0.390594
XGBoost_1_AutoML_22_20231127_01829                       0.62558   0.39135   0.445292  0.148768                  0.39135
DRF_1_AutoML_22_20231127_01829                           0.626148  0.392062  0.44421   0.148858                  0.392062
XGBoost_2_AutoML_22_20231

In [202]:
test_h2o = h2o.H2OFrame(X_test)
best_model = aml.leader

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [203]:
pred = best_model.predict(test_h2o)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [204]:
import numpy as np

# y_test = y_test.to_numpy()
pred = pred.as_data_frame()
pred = pred.to_numpy()

In [205]:
mse = mean_squared_error(y_test, pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.3728193124620351


In [176]:
Xtrain = TOX.drop(columns=['mouse intraperitoneal LD50'])
ytrain = TOX['mouse intraperitoneal LD50']

# Split the data into training (80%), and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(Xtrain, ytrain, test_size=0.2, random_state=42)

In [177]:
# Gradient Boosting Machine (GBM)
from sklearn.ensemble import GradientBoostingRegressor  # For regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

gbm_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbm_regressor.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

In [178]:
y_pred = gbm_regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.40154546555599063


### Feature Selection

### Classification

In [98]:
TOX_group = pd.read_csv("interaction_data.csv")
TOX_group['group'], unique = pd.factorize(TOX_group['group'])

In [99]:
TOX_group.head()

,Halogens,S,P,Other Metal,Hydroxyl or Ether,"Ketones, aldehydes, carboxylic acids, or esters",Primary Amine or Ammonium Ion,Secondary Amine or Aromatic Nitrogen,Tertiary Amine or Amide Nitrogen,Aromatic,...,Primary_Amine_Ketones_aldehydes_acids_esters,Secondary_Amine_Ketones_aldehydes_acids_esters,Tertiary_Amine_Ketones_aldehydes_acids_esters,Primary_Amine_Hydroxyl_or_Ether,Secondary_Amine_Hydroxyl_or_Ether,Tertiary_Amine_Hydroxyl_or_Ether,Aromatic_Hydroxyl_or_Ether,Aromatic_Other_Metal,Hydroxyl_or_Ether_Ketones_aldehydes_acids_esters,Aliphatic_Tertiary_Amine_or_Amide_Nitrogen
0,0,0,0,0,1,-1,0,0,0,1,...,0,0,0,0,0,0,1,0,-1,0
1,0,1,1,0,0,0,-1,-1,1,0,...,0,0,0,0,0,0,0,0,0,1
2,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,-1,0,0,0,1,...,0,0,0,0,0,0,1,0,-1,0


In [139]:
Xtrain = TOX_group.drop(columns=['mouse intraperitoneal LD50',
                           'group'])
ytrain = TOX_group['group']

# Split the data into training (70%), validation (15%), and test (15%) sets
X_train, X_temp, y_train, y_temp = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [140]:
# H2O.ai
h2o.init()

Xtrain_h2o = h2o.H2OFrame(X_train)
Xtest_h2o = h2o.H2OFrame(X_test)
X_valid_h2o = h2o.H2OFrame(X_valid)
ytrain_h2o = h2o.H2OFrame(list(y_train), column_names=["group"])
ytest_h2o = h2o.H2OFrame(list(y_test), column_names=["group"])
yvalid_h2o = h2o.H2OFrame(list(y_valid), column_names=["group"])


ytrain_h2o['group'] = ytrain_h2o['group'].asfactor()
ytest_h2o['group'] = ytest_h2o['group'].asfactor()
yvalid_h2o['group'] = yvalid_h2o['group'].asfactor()


train_h2o = Xtrain_h2o.cbind(ytrain_h2o)
test_h2o = Xtest_h2o.cbind(ytest_h2o)
valid_h2o = X_valid_h2o.cbind(yvalid_h2o)

aml = H2OAutoML(max_runtime_secs=600, max_models=10, seed=42, nfolds=5)
aml.train(y="group", training_frame=train_h2o, validation_frame=valid_h2o)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,2 hours 29 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_unknownUser_lvn68v
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.751 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
23:39:13.380: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_2_AutoML_15_20231126_233913


Model Summary: 
    number_of_trees
--  -----------------
    39

ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.3950575495134094
RMSE: 0.6285360367659195
LogLoss: 1.0900968638104955
Mean Per-Class Error: 0.6346205699590735
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
0      1     2     3     4    Error     Rate
-----  ----  ----  ----  ---  --------  ---------------
9066   189   1367  502   30   0.187197  2,088 / 11,154
566    576   358   95    16   0.642458  1,035 / 1,611
3573   243   2507  207   19   0.617193  4,042 / 6,549
3237   97    348   904   23   0.803862  3,705 / 4,609
497    47    63    118   61   0.922392  725 / 786
16939  1152  4643  1826  149  0.469262  11,595 / 24,709

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.530738
2    0.814723
3    0.9416
4    0.984985
5    1

ModelMetricsMultinomial: xgboost
** Reported on validation data. **

MSE: 0.41202717772201414
RMSE: 0.6418934317486152
LogLoss: 1.147123912409815
Mean Per-Class Error: 0.6677902014748351
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
0     1    2    3    4    Error     Rate
----  ---  ---  ---  ---  --------  -------------
1903  41   315  140  9    0.209718  505 / 2,408
125   104  73   19   1    0.677019  218 / 322
837   58   478  47   2    0.663854  944 / 1,422
731   20   72   157  14   0.842052  837 / 994
88    10   13   30   8    0.946309  141 / 149
3684  233  951  393  34   0.499528  2,645 / 5,295

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.500472
2    0.802833
3    0.934466
4    0.980359
5    1

ModelMetricsMultinomial: xgboost
** Reported on cross-validation data. **

MSE: 0.41652271902930915
RMSE: 0.6453857133755823
LogLoss: 1.166666704783424
Mean Per-Class Error: 0.6616359391007764
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
0      1     2     3     4    Error     Rate
-----  ----  ----  ----  ---  --------  ---------------
8754   203   1507  643   47   0.215169  2,400 / 11,154
603    502   387   99    20   0.688392  1,109 / 1,611
3813   267   2220  220   29   0.661017  4,329 / 6,549
3335   107   337   783   47   0.830115  3,826 / 4,609
486    53    53    126   68   0.913486  718 / 786
16991  1132  4504  1871  211  0.501113  12,382 / 24,709

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.498887
2    0.790238
3    0.928285
4    0.977822
5    1

Cross-Validation Metrics Summary: 
                         mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
-----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------
accuracy                 0.498887  0.00428056  0.491704      0.498584      0.502226      0.5           0.501923
auc                      nan       0           nan           nan           nan           nan   

In [141]:
# View AutoML leaderboard
leaderboard = aml.leaderboard
print(leaderboard)

model_id                               mean_per_class_error    logloss      rmse       mse
XGBoost_2_AutoML_15_20231126_233913                0.661636    1.16667  0.645386  0.416523
XGBoost_1_AutoML_15_20231126_233913                0.663712    1.16508  0.646189  0.41756
GBM_1_AutoML_15_20231126_233913                    0.678469    1.16446  0.64967   0.422072
GLM_1_AutoML_15_20231126_233913                    0.769114    1.24822  0.67909   0.461163
[4 rows x 5 columns]



In [142]:
test_h2o = h2o.H2OFrame(X_test)
best_model = aml.leader

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [143]:
pred = best_model.predict(test_h2o)

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [144]:
pred.tail()

predict,p0,p1,p2,p3,p4
0,0.556718,0.00973462,0.245634,0.172646,0.0152668
2,0.441631,0.00783113,0.454323,0.0348115,0.0614034
4,0.228715,0.130084,0.201588,0.111827,0.327786
2,0.0384834,0.0377941,0.905054,0.0146854,0.00398306
2,0.450986,0.0299015,0.478814,0.0347945,0.0055042
2,0.341188,0.0194808,0.412521,0.212018,0.014792
2,0.342264,0.205784,0.375166,0.0351168,0.0416695
0,0.741056,0.00416421,0.167676,0.0858669,0.00123681
0,0.545623,0.0417188,0.17672,0.152679,0.0832594
2,0.279389,0.0263068,0.662956,0.0246691,0.00667873


In [145]:
predictions_df = h2o.as_list(pred['predict'], use_pandas=True)
predictions_list = predictions_df['predict'].tolist()

In [146]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions_list)
print(accuracy)

0.497261567516525
